<a href="https://colab.research.google.com/github/max-broekhuis/workshop-consultancy-day/blob/main/code_ec_energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Inladen Data



In [2]:
!curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id=1mlqgfqfBmp9Att1KHQZADannFgAqt7l6' | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt
!curl -L -b cookies.txt -o 'EC_Energy_case.xlsx' 'https://docs.google.com/uc?export=download&id=1mlqgfqfBmp9Att1KHQZADannFgAqt7l6&confirm='$(<confirm.txt)
!rm -f confirm.txt cookies.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    285      0 --:--:--  0:00:01 --:--:--   284
100 5257k    0 5257k    0     0  2598k      0 --:--:--  0:00:02 --:--:-- 2598k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    323      0 --:--:--  0:00:01 --:--:--   323
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 5257k    0 5257k    0     0  3486k      0 --:--:--  0:00:01 --:--:-- 3486k


In [3]:
data = pd.read_excel("EC_Energy_case.xlsx")
data = data.assign(totaal_zonnepanelen = lambda x: x.aantal_installaties_bij_woningen*x.aantal_zonnepanelen_per_installatie)

# Analyse data

In [4]:
data.head()

,X1,gemeentenaam,soort_regio,id,codering,aantal_inwoners,woningkenmerken,stadsverwarming,mannen,vrouwen,k_0_tot_15_jaar,k_15_tot_25_jaar,k_25_tot_45_jaar,k_45_tot_65_jaar,k_65_jaar_of_ouder,huishoudens_totaal,eenpersoonshuishoudens,huishoudens_zonder_kinderen,huishoudens_met_kinderen,gemiddelde_huishoudensgrootte,bevolkingsdichtheid,woningvoorraad,gemiddelde_woningwaarde,koopwoningen,huurwoningen_totaal,in_bezit_woningcorporatie,in_bezit_overige_verhuurders,eigendom_onbekend,bouwjaar_voor_2000,bouwjaar_vanaf_2000,gemiddeld_elektriciteitsverbruik_totaal,elect_appartement,elect_tussenwoning,elect_hoekwoning,elect_twee_onder_een_kap_woning,elect_vrijstaande_woning,elect_huurwoning,elect_eigen_woning,gemiddeld_aardgasverbruik_totaal,aard_appartement,aard_tussenwoning,aard_hoekwoning,aard_twee_onder_een_kap_woning,aard_vrijstaande_woning,aard_huurwoning,aard_eigen_woning,percentage_woningen_met_stadsverwarming,aantal_inkomensontvangers,gemiddeld_inkomen_per_inkomensontvanger,gemiddeld_inkomen_per_inwoner,k_40_huishoudens_met_laagste_inkomen,k_20_huishoudens_met_hoogste_inkomen,personen_per_soort_uitkering_bijstand,personenautos_brandstof_benzine,personenautos_overige_brandstof,oppervlakte_land,mate_van_stedelijkheid,omgevingsadressendichtheid,bedrijfsvestigingen_totaal,type_a_landbouw_bosbouw_visserij,type_bf_nijverheid_energie,type_gi_handel_horeca,type_hj_vervoer_informatie_communicatie,type_kl_financiele_diensten_onroerendgoed,type_mn_zakelijke_dienstverlening,type_ru_cultuur_recreatie_overige_diensten,id_old,indelingswijziging_wijken_en_buurten,aantal_installaties_bij_woningen,aantal_zonnepanelen_per_installatie,opgesteld_vermogen_van_zonnepanelen,totaal_aantal_laadpalen,werkloosheidsuitkering_relatief,bijstandsuitkering_relatief,arbeidsongeschiktheidsuitkering_relatief,aow_uitkering_relatief,inwoners_vanaf_15_jaar,inwoners_vanaf_15_jr_tot_aow_leeftijd,inwoners_vanaf_de_aow_leeftijd,totaal_zonnepanelen
0,9503,LoonopZand,Buurt,96,BU08093091,222,T001100ZW25810ZW25805ZW25806ZW10300ZW103201014...,0,110,110,35,30,40,70,45,80,15,35,35,26.0,26.0,92,425.0,90.0,8.0,0.0,8.0,2.0,74.0,26.0,4680.0,NaN,1900.0,NaN,4000.0,5420.0,NaN,4550.0,2350.0,NaN,910.0,NaN,1630.0,2860.0,NaN,2190.0,NaN,200,NaN,NaN,NaN,NaN,0.0,90,35,856,5.0,235.0,35.0,10.0,5.0,10.0,0.0,5.0,5.0,0.0,1004,1.0,10.0,26.0,60.0,0.0,90,20,155,930,800,630,200,260.0
1,4063,Dordrecht,Buurt,67,BU05050202,1046,T001100ZW25810ZW25805ZW25806ZW10300ZW103201014...,0,535,505,200,155,285,290,115,450,175,95,190,23.0,5979.0,437,131.0,24.0,76.0,71.0,5.0,0.0,91.0,9.0,2470.0,2050.0,3080.0,2940.0,NaN,NaN,2220.0,3280.0,840.0,720.0,970.0,980.0,NaN,NaN,790.0,980.0,NaN,800,NaN,NaN,607.0,76.0,110.0,235,70,17,1.0,2972.0,50.0,0.0,5.0,10.0,10.0,5.0,5.0,15.0,1006,1.0,NaN,NaN,NaN,0.0,90,485,455,520,3380,2950,440,NaN
2,2473,hetBildt,Wijk,96,WK006301,1677,T001100ZW25810ZW25805ZW25806ZW10300ZW103201014...,0,885,790,290,205,365,560,255,725,240,225,260,23.0,59.0,750,139.0,76.0,23.0,15.0,8.0,1.0,94.0,6.0,2690.0,NaN,1950.0,2380.0,2540.0,2930.0,1990.0,2900.0,1600.0,NaN,1300.0,1390.0,1420.0,1770.0,1330.0,1690.0,NaN,1300,NaN,NaN,363.0,163.0,30.0,640,240,2843,5.0,80.0,180.0,40.0,30.0,35.0,15.0,10.0,35.0,25.0,1000,1.0,70.0,18.0,278.0,2.0,125,80,185,710,5570,4560,1000,1260.0
3,11554,Oudewater,Buurt,42,BU05890208,134,T001100ZW25810ZW25805ZW25806ZW10300ZW103201014...,0,70,60,25,20,25,50,10,45,10,15,25,30.0,436.0,49,NaN,84.0,14.0,0.0,14.0,2.0,61.0,39.0,5120.0,NaN,NaN,NaN,5030.0,5450.0,NaN,4830.0,2210.0,NaN,NaN,NaN,2270.0,2410.0,NaN,2050.0,NaN,100,NaN,NaN,NaN,NaN,0.0,90,85,31,4.0,535.0,120.0,0.0,40.0,45.0,10.0,20.0,5.0,0.0,1008,1.0,6.0,11.0,14.0,1.0,10,0,40,375,440,400,40,66.0
4,6943,Heerlen,Buurt,16,BU09173201,340,T001100ZW25810ZW25805ZW25806ZW10300ZW103201014...,0,150,185,20,25,55,90,140,255,195,35,25,13.0,1542.0,271,81.0,11.0,66.0,30.0,37.0,23.0,100.0,0.0,2040.0,1840.0,2490.0,2210.0,NaN,NaN,2120.0,1940.0,930.0,850.0,1050.0,1600.0,NaN,NaN,960.0,930.0,NaN,300,NaN,NaN,879.0,0.0,30.0,90,5,22,2.0,1763.0,0.0,NaN,NaN,NaN,NaN,NaN,N

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16666 entries, 0 to 16665
Data columns (total 80 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   X1                                          16666 non-null  int64  
 1   gemeentenaam                                16666 non-null  object 
 2   soort_regio                                 16666 non-null  object 
 3   id                                          16666 non-null  int64  
 4   codering                                    16666 non-null  object 
 5   aantal_inwoners                             16666 non-null  int64  
 6   woningkenmerken                             16666 non-null  object 
 7   stadsverwarming                             16666 non-null  int64  
 8   mannen                                      16666 non-null  int64  
 9   vrouwen                                     16666 non-null  int64  
 10  k_0_tot_15

In [6]:
data["gemeentenaam"].nunique()

388

In [7]:
data["soort_regio"].value_counts()

Buurt       13208
Wijk         3070
Gemeente      388
Name: soort_regio, dtype: int64

In [8]:
 (
    data
    .groupby(["gemeentenaam"])
    ["totaal_aantal_laadpalen"]
    .mean()
    .sort_values(ascending = False)
    .head(500)
)


gemeentenaam
Langedijk          17.250000
Zeewolde           12.538462
Culemborg          12.100000
Oldenzaal          10.038462
StedeBroec          9.250000
                     ...    
Zwijndrecht         0.217391
Zandvoort           0.176471
Terschelling        0.150000
Wassenaar           0.136364
Schiermonnikoog     0.000000
Name: totaal_aantal_laadpalen, Length: 388, dtype: float64

In [9]:
(
    data
    .groupby(["gemeentenaam"])
    ["totaal_zonnepanelen"]
    .mean()
    .sort_values(ascending = False)
    .head(500)
)

gemeentenaam
Langedijk              7193.428571
Zeewolde               5560.000000
Appingedam             5319.250000
GilzeenRijen           4757.000000
Beek                   4677.571429
                          ...     
Maassluis               468.166667
Vlieland                444.000000
Schiermonnikoog         418.000000
CapelleaandenIJssel     354.076923
Schiedam                329.939394
Name: totaal_zonnepanelen, Length: 388, dtype: float64

# Regressie

Omdat EC Energy intern niet veel werknemers heeft met verstand van Data Science is er een stagiaire gevraagd om een opzet te maken van een model.

Deze stagiaire heeft de data gefiltered op een aantal interessante variabelen. Daarna zijn deze variabelen door een lineaire regressie gehaald. Op basis van de resultaten is EC Energy enthousiast, maar geeft de stagiaire aan dat er problemen zijn met deze analyse en een verdieping noodzakelijk is. Daarom heeft EC Energy meerdere consultancy partijen gevraag een business case op te stellen.

In [14]:
data['EV_totaal'] = data['personenautos_overige_brandstof'] * 0.2333
X = (data[
          [
           "EV_totaal",
           "huishoudens_totaal",
           "totaal_aantal_laadpalen"
           ]
          ])
X.dropna(inplace =True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [15]:
x = X.drop("totaal_aantal_laadpalen", axis=1)
y = X["totaal_aantal_laadpalen"]
x = sm.add_constant(x)
model = sm.OLS(y, x)
results = model.fit()
print(results.summary())

                               OLS Regression Results                              
Dep. Variable:     totaal_aantal_laadpalen   R-squared:                       0.461
Model:                                 OLS   Adj. R-squared:                  0.461
Method:                      Least Squares   F-statistic:                     6260.
Date:                     Tue, 15 Jun 2021   Prob (F-statistic):               0.00
Time:                             09:28:45   Log-Likelihood:                -49426.
No. Observations:                    14622   AIC:                         9.886e+04
Df Residuals:                        14619   BIC:                         9.888e+04
Df Model:                                2                                         
Covariance Type:                 nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------

# Vraagstelling

Na het zien van deze regressie is EC Energy benieuwd wat er nog meer mogelijk is met deze data. Daarom hebben ze jou gevraagd om een business case op te zetten. Hierin wilt EC Energy een duidelijk advies waar ze hun salesstrategie op moeten toespitsen. Ze laten jou vrij om iedere techniek te gebruiken waarvan jij denkt dat het het meeste resultaat gaat opleveren.

EC Energy gaat jou beoordelen op hoe begrijpbaar jouw salesstrategie is (50%), hoe goed je model/tool werkt (20%) en hoe dit zich verhoudt tot je concurrenten (30%).